In [ ]:
# 학습/테스트 데이터 셋 분리 – train_test_split()
# 첫번째 파라미터로 피처 데이터 세트, 두번째 파라미터로 레이블 데이터 세트를 입력 받는다.
# 선택적 파라미터로 test_size(디폴트는 0.25), shuffle(디폴트는 True), random_state를 입력 받는다.
# train_test_split()의 반환값은 튜플 형태이다.

In [1]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np

In [2]:
# 학습 데이터 세트로만 학습하고 예측
iris = load_iris()
dt_clf = DecisionTreeClassifier()
train_data = iris.data
train_label = iris.target

# 독립변수, 종속변수 준비
train_data = iris.data
train_label = iris.target

# 학습
dt_clf.fit(train_data, train_label)

# 예측
pred = dt_clf.predict(train_data)
print('예측 정확도:', accuracy_score(train_label, pred))

예측 정확도: 1.0


In [ ]:
# X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.25(default), random_state=42)

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target,
                                                   test_size=0.3, random_state=121)
dt_clf.fit(X_train,y_train)
pred = dt_clf.predict(X_test)
print("예측 정확도:{0:.4f}".format(accuracy_score(y_test,pred)))

예측 정확도:0.9556


In [ ]:
# 교차 검증(Cross-Validation)
# 학습용 데이터와 테스트용 데이터를 분리해서 모델링 및 평가를 수행하는 방식은 '과적합'에 취약한 약점을 가질 수 있다.
# 과적합은 모델이 학습 데이터에만 과도하게 최적화되어 실제 예측을 다른 데이터로 수행할 때 예측 성능이 과도하게 떨어지는 것을 의미한다.
# 고정된 테스트 데이터로만 평가를 하게되면 테스트 데이터에만 최적의 성능을 발휘할 수 있도록 편향되게 모델을 유도하는 경향이 발생한다.
# 이러한 문제점을 개선하기 위하여 '교차 검증'을 이용해 다양한 학습과 평가를 수행한다.

In [ ]:
# K 폴드
# K개의 데이터 폴드 세트를 만들어서 K번만큼 각 폴드 세트에 학습과 검증 평가를 반복적으로 수행하는 방법
# 데이터 세트를 K등분, 학습데이터와 검증 데이터 세트를 K번 변경하면서 
# 학습과 검증을 수행한 결과를 평균해서 평가 결과 산출
# 사이킷런은 KFold와 StratifiedKFold 클래스를 제공

In [4]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import numpy as np

iris = load_iris()
iris.keys()
features = iris.data
label = iris.target
dt_clf = DecisionTreeClassifier(random_state=156)

kfold = KFold(n_splits=5)
cv_accuracy=[]
n_iter=0
for train_index,test_index in kfold.split(features):
    # kfold.split()으로 반환된 인덱스를 이용하여 학습용, 검증용 테스트 데이터 추출
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]
    # 학습 및 예측
    dt_clf.fit(X_train, y_train)
    pred = dt_clf.predict(X_test)
    n_iter += 1
    accuracy = np.round(accuracy_score(y_test,pred),4)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    print('\n#{0} 교차 검증 정확도 :{1}, 학습 데이터 크기: {2},검증 데이터 크기: {3}'\
          .format(n_iter, accuracy, train_size, test_size))
    cv_accuracy.append(accuracy)
# 개별 iteration별 정확도를 합하여 평균 정확도 계산
print('\n## 평균 검증 정확도:', np.mean(cv_accuracy))


#1 교차 검증 정확도 :1.0, 학습 데이터 크기: 120,검증 데이터 크기: 30

#2 교차 검증 정확도 :0.9667, 학습 데이터 크기: 120,검증 데이터 크기: 30

#3 교차 검증 정확도 :0.8667, 학습 데이터 크기: 120,검증 데이터 크기: 30

#4 교차 검증 정확도 :0.9333, 학습 데이터 크기: 120,검증 데이터 크기: 30

#5 교차 검증 정확도 :0.7333, 학습 데이터 크기: 120,검증 데이터 크기: 30

## 평균 검증 정확도: 0.9


In [ ]:
# Stratified K 폴드
# 불균형한 분포도를 가진 레이블 데이터 집합을 위한 K폴드 방식
# 불균형한 분포도를 가진 레이블 데이터 집합은 특정 레이블 값이 특이하게 많거나 
# 매우 적어서 값의 분포가 한쪽으로 치우치는 것을 의미한다. 
# 대출 사기 데이터를 예측하는 경우 대부분의 데이터가 정상 데이터이므로 대출 사기 레이블의 비율이 아주 적게 될 수 있으며 레이블 값 1이 특정 개별 반복별 학습/테스트 데이터 세트에는 상대적으로 많이 들어가고 다른 데이터 세트에는 그렇지 않을 수가 있다.
# 대출 사기 레이블이 1인 레코드는 비록 건수는 작지만 알고리즘이 대출 사기를 예측하기 위한 중요한 피처를 가지고 있으므로 중요한 데이터 세트로서 원본 데이터와 유사한 대출 사기 레이블 값의 분포를 학습/테스트 세트에도 유지하는게 중요하다.
# Stratified KFold는 KFold가 레이블 데이터 집합이 원본 데이터 집합의 레이블 분포를 학습 및 테스트 세트에 제대로 분배하지 못하는 경우의 문제를 해결해 준다.
# 일반적으로 분류에서의 교차 검증은 Stratified KFold로 분할되어야 한다.
# 회귀의 결정값은 이산값 형태의 레이블이 아니라 연속된 숫자값이기 때문에 결정값별로 분포를 정하는 것이 의미가 없으므로 Stratified KFold가 지원되지 않는다.

In [5]:
import pandas as pd
iris = load_iris()
type(iris.data)
iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_df['label'] = iris.target
iris_df.head()
iris_df.label.value_counts()

2    50
1    50
0    50
Name: label, dtype: int64

In [7]:
iris.keys()

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names', 'filename'])

In [9]:
iris.target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [13]:
iris_df = pd.DataFrame(data=train_data, columns=iris.feature_names)
iris_df['label'] = iris.target
iris_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),label
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [18]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

# 독립변수, 종속변수 분리
y = iris_df['label']
X = iris_df.drop('label',axis=1)
# 학습용 테이터와 평가용 데이터를 8:2로 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10) 

In [14]:
kfold = KFold(n_splits=3)
n_iter=0
for train_index, test_index in kfold.split(iris_df):
    n_iter += 1
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]
    print('## 교차 검증: {0}'.format(n_iter))
    print('학습 레이블 데이터 분포:\n', label_train.value_counts())
    print('검증 레이블 데이터 분포:\n', label_test.value_counts())

## 교차 검증: 1
학습 레이블 데이터 분포:
 2    50
1    50
Name: label, dtype: int64
검증 레이블 데이터 분포:
 0    50
Name: label, dtype: int64
## 교차 검증: 2
학습 레이블 데이터 분포:
 2    50
0    50
Name: label, dtype: int64
검증 레이블 데이터 분포:
 1    50
Name: label, dtype: int64
## 교차 검증: 3
학습 레이블 데이터 분포:
 1    50
0    50
Name: label, dtype: int64
검증 레이블 데이터 분포:
 2    50
Name: label, dtype: int64


In [15]:
# Stratified KFold는 원본 데이터의 레이블 분포를 먼저 고려한 뒤,
#이 분포와 동일하게 학습과 테스트 세트를 분배

from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=3)
n_iter=0
for train_index, test_index in skf.split(iris_df, iris_df['label']):
    n_iter += 1
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]
    print('## 교차 검증: {0}'.format(n_iter))
    print('학습 레이블 데이터 분포:\n', label_train.value_counts())
    print('검증 레이블 데이터 분포:\n', label_test.value_counts())

## 교차 검증: 1
학습 레이블 데이터 분포:
 2    34
1    33
0    33
Name: label, dtype: int64
검증 레이블 데이터 분포:
 1    17
0    17
2    16
Name: label, dtype: int64
## 교차 검증: 2
학습 레이블 데이터 분포:
 1    34
2    33
0    33
Name: label, dtype: int64
검증 레이블 데이터 분포:
 2    17
0    17
1    16
Name: label, dtype: int64
## 교차 검증: 3
학습 레이블 데이터 분포:
 0    34
2    33
1    33
Name: label, dtype: int64
검증 레이블 데이터 분포:
 2    17
1    17
0    16
Name: label, dtype: int64


In [ ]:
# cross_val_score( )
# 사이킷런이 제공하는 교차검증을 좀 더 편리하게 수행할 수 있는 대표적이 API이다.
# KFold의 일련의 과정을 한꺼번에 수행해준다.
# 주요 파라미터는 estimator, X, y, scoring, cv이다.
# cross_val_score()는 내부적으로 StratifiedKFold를 이용한다.
# cross_val_score()는 하나의 평가 지표만 가능하나,
# cross_validate()는 여러 개의 평가 지표를 반환한다.

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html?highlight=cross%20val%20score

In [16]:
from sklearn.model_selection import cross_val_score
iris_data = load_iris()
dt_clf=DecisionTreeClassifier(random_state=156)
data=iris_data.data
label=iris_data.target
scores = cross_val_score(dt_clf, data, label, scoring='accuracy',cv=3)
print('교차 검증별 정확도:',np.round(scores,4))
print('평균 검증 정확도:',np.round(np.mean(scores),4))

교차 검증별 정확도: [0.98 0.94 0.98]
평균 검증 정확도: 0.9667


In [ ]:
# GridSearchCV
# 하이퍼 파라미터는 ML 알고리즘의 주요 구성 요소이며,
# 이 값을 조정해 알고리즘의 예측 성능을 개선할 수 있다.
# 사이킷런은 GridSearchCV API를 이용해 Classifier나 Regressor와 같은 알고리즘에 
# 사용되는 하이퍼 파라미터를 순차적으로 입력하면서 편리하게 최적의 파라미터를 도출할 수 있는 방안을 제공한다.
# GridSearchCV는 교차 검증을 기반으로 이 하이퍼 파라미터의 최적 값을 찾게 해준다. 
# 즉 데이터 세트를 cross_validation을 위한 학습/테스트 세트로 
# 자동 분할한 뒤에 하이퍼 파라미터 그리드에 기술된 모든 파라미터를 순차적으로 적용해 최적의 파라미터를 찾을 수 있게 해준다.
# CV가 3회, 6개의 파라미터 조합이라면 총 18회의 학습/평가가 이루어진다.
# 주요 파라미터 : estimator, param_grid, scoring, cv, refit
# param_grid : estimator 튜닝을 위해 파라미터명과 사용될 여러 파라미터 값을 지정
# refit : 디폴트가 True이며 True로 생성 시 가장 최적의 하이퍼 파라미터를 찾은 뒤 
# 입력된 estimator 객체를 해당 하이퍼 파라미터로 재학습시킨다.

In [18]:
from sklearn.model_selection import GridSearchCV

iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris_data.data, iris_data.target,
                                                   test_size=0.2, random_state=121)
dt_clf = DecisionTreeClassifier()
parameters = {'max_depth':[1,2,3], 'min_samples_split':[2,3]}

grid_dtree = GridSearchCV(dt_clf, param_grid=parameters, cv=3, refit=True)
grid_dtree.fit(X_train,y_train)

print('GridSearchCV 최적 파라미터:',grid_dtree.best_params_)
print('GridSearchCV 최고 정확도: {0:4f}'.format(grid_dtree.best_score_))

estimator = grid_dtree.best_estimator_
pred = estimator.predict(X_test)
print('테스트 데이터 세트 정확도: {0:.4f}'.format(accuracy_score(y_test,pred)))

GridSearchCV 최적 파라미터: {'max_depth': 3, 'min_samples_split': 2}
GridSearchCV 최고 정확도: 0.975000
테스트 데이터 세트 정확도: 0.9667


In [6]:
# cross_val_score
from sklearn.model_selection import cross_val_score

In [9]:
iris = load_iris()
iris.keys()
features = iris.data
label = iris.target
dt_clf = DecisionTreeClassifier(random_state=156)

In [10]:
scores = cross_val_score(dt_clf, features, label, scoring='accuracy',cv=3)
print('교차 검증별 정확도:', np.round(scores,2))
print('평균 검증 정확도:', np.round(np.mean(scores),2))

교차 검증별 정확도: [0.98 0.94 0.98]
평균 검증 정확도: 0.97


In [19]:
from sklearn.model_selection import GridSearchCV
parameters = {'max_depth': [2,3,5,10], 'min_samples_split':[2,3,5], 
             'min_samples_leaf':[1,5,8]}

# cv = cross validation의 약자
grid_dclf = GridSearchCV(dt_clf, param_grid=parameters, scoring='accuracy', cv=3)
grid_dclf.fit(X_train, y_train)
print('GCV 최적 하이퍼 파라미터', grid_dclf.best_params_)
print('GCV 최고 정확도:{0: .4f}'.format(grid_dclf.best_score_))
best_dclf = grid_dclf.best_estimator_

# GridSearchCV의 최적 하이퍼 파라미터로 학습된 Estimator로 예측 및 평가 수행
dpred = best_dclf.predict(X_test)
accuracy = accuracy_score(y_test, dpred)
print('DT 정확도 {0: 4f}'.format(accuracy))

GCV 최적 하이퍼 파라미터 {'max_depth': 2, 'min_samples_leaf': 1, 'min_samples_split': 2}
GCV 최고 정확도: 0.9238
DT 정확도  0.977778
